In [16]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# -------------- Dataset Definition --------------
class SequenceDataset(Dataset):
    def __init__(self, file_paths, labels, n_mfcc=None):
        self.file_paths = file_paths
        self.labels = labels
        self.n_mfcc = n_mfcc

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        arr = np.load(path)
        # Handle MFCC reshape
        if self.n_mfcc and arr.ndim == 1:
            T = arr.size // self.n_mfcc
            arr = arr.reshape(self.n_mfcc, T)
        # arr now shape (feat_dim, time_steps)
        # for LSTM: (time_steps, feat_dim)
        x = torch.tensor(arr, dtype=torch.float32).transpose(0, 1)
        # filter label
        y_filter = 0 if 'unfiltered' in path else 1
        # downstream class label from filename (number before .npy)
        fname = os.path.basename(path)
        class_id = int(fname.split('-')[1].split(".")[0])
        y_class = torch.tensor(class_id, dtype=torch.long)
        return x, y_class, y_filter

# -------------- Model Definitions --------------
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, num_classes=10):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x: (B, T, D)
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1])
        return out

class CNNClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # conv stack
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2)
        # instead of hard-coding dims, force output to 1×1
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

        # now every sample is reduced to a (B, 64, 1, 1) tensor → 64 features
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # x: (B, T, D) → (B, 1, D, T)
        x = x.permute(0, 2, 1).unsqueeze(1)
        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        # global pooling
        x = self.adaptive_pool(x)         # → (B, 64, 1, 1)
        x = x.view(x.size(0), -1)         # → (B, 64)
        x = F.relu(self.fc1(x))           # → (B, 128)
        return self.fc2(x)                # → (B, num_classes)

# -------------- Training & Evaluation --------------
def train_model(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for x, y_class, _ in tqdm(loader, desc="Training"):
        x, y = x.to(device), y_class.to(device)
        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x.size(0)
    print("Running Loss", running_loss/len(loader))
    return running_loss / len(loader.dataset)

@torch.no_grad()
def evaluate_model(model, loader, device):
    model.eval()
    all_preds, all_labels, all_filters = [], [], []
    for x, y_class, y_filter in tqdm(loader, desc="Eval"):
        x = x.to(device)
        logits = model(x)
        preds = logits.argmax(dim=1).cpu()
        all_preds.append(preds)
        all_labels.append(y_class)
        all_filters.append(y_filter)
    y_pred = torch.cat(all_preds)
    y_true = torch.cat(all_labels)
    y_filt = torch.cat(all_filters)
    overall_acc = (y_pred == y_true).float().mean().item()
    # per subset
    filt_metrics = {}
    for val, name in [(0, 'unfiltered'), (1, 'filtered')]:
        mask = (y_filt == val)
        if mask.sum() > 0:
            filt_metrics[name] = (y_pred[mask] == y_true[mask]).float().mean().item()
        else:
            filt_metrics[name] = None
    return overall_acc, filt_metrics

# -------------- Main Experiment Loop --------------
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    DATASETS = {
        'STFT': [r"D:\Aliasing3\Processed_Files\DS_ESC",
                 r"D:\Aliasing3\Processed_Files\DS_U8K"],
        'MFCC': [r"D:\Aliasing3\Processed_Files\DS_ESC_MFCC",
                 r"D:\Aliasing3\Processed_Files\DS_U8K_MFCC"]
    }
    frequencies = [1000, 2000, 4000, 8000, 16000, 22000]
    splits = ['train', 'validation', 'test']
    batch_size = 4
    epochs = 50
    results = []
    for feat_type, roots in DATASETS.items():
        for root in roots:
            for freq in frequencies:
                for model_name in ['LSTM', 'CNN']:
                    # collect file paths
                    paths = {s: {'filtered': [], 'unfiltered': []} for s in splits}
                    for split in splits:
                        for label in ['filtered', 'unfiltered']:
                            folder = os.path.join(root, str(freq), split, label)
                            paths[split][label] = glob.glob(os.path.join(folder, '*.npy'))
                    # build datasets & loaders
                    train_files = paths['train']['filtered'] + paths['train']['unfiltered']
                    train_labels = [None]*len(train_files)  # class label parsed later
                    val_files   = paths['validation']['filtered'] + paths['validation']['unfiltered']
                    val_labels  = [None]*len(val_files)
                    test_files   = paths['test']['filtered'] + paths['test']['unfiltered']
                    test_labels  = [None]*len(test_files)
                    n_mfcc = 20 if feat_type == 'MFCC' else None

                    print("Train  filtered:", len(paths['train']['filtered']),
                          "unfiltered:", len(paths['train']['unfiltered']))
                    print("Valid  filtered:", len(paths['validation']['filtered']),
                          "unfiltered:", len(paths['validation']['unfiltered']))
                    
                    print("Total training files found:", len(train_files))
                    print("Total val files found:", len(val_files))
                    
                    train_ds = SequenceDataset(train_files, train_labels, n_mfcc)
                    val_ds   = SequenceDataset(val_files, val_labels, n_mfcc)
                    test_ds   = SequenceDataset(test_files, test_labels, n_mfcc)
                    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
                    val_loader   = DataLoader(val_ds, batch_size=batch_size)
                    test_loader   = DataLoader(test_ds, batch_size=batch_size)
                    

                    # instantiate model
                    # infer input_dim from a sample
                    sample_x, _, _ = next(iter(train_loader))
                    input_dim = sample_x.shape[2]
                    num_classes = 50 if "ESC" in root else 10
                    if model_name == 'LSTM':
                        model = LSTMClassifier(input_dim, hidden_dim=128, num_layers=3, num_classes=num_classes)
                    else:
                        model = CNNClassifier(num_classes=num_classes)
                    model.to(device)

                    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
                    criterion = nn.CrossEntropyLoss()

                    # right before your epoch loop, initialize trackers
                    best_val_acc   = 0.0
                    best_epoch     = 0
                    best_state     = None
                    epoch_records  = []
                    
                    # make sure you have a directory to save checkpoints
                    os.makedirs('checkpoints', exist_ok=True)
                    
                    for epoch in range(1, epochs+1):
                        # 1) train one epoch
                        train_loss = train_model(model, train_loader, criterion, optimizer, device)
                    
                        # 2) eval on validation
                        val_acc, val_metrics = evaluate_model(model, val_loader, device)
                        print(f"Epoch {epoch} — val_acc: {val_acc:.3%}")
                    
                        # 3) record for later analysis
                        epoch_records.append({
                            'epoch':      epoch,
                            'val_acc':    val_acc,
                            'unfilt_acc': val_metrics['unfiltered'],
                            'filt_acc':   val_metrics['filtered']
                        })
                    
                        # 4) save a checkpoint for *this* epoch
                        ckpt_path = f'checkpoints/{model_name}_{feat_type}_{freq}Hz_epoch{epoch:02d}.pt'
                        torch.save(model.state_dict(), ckpt_path)
                    
                        # 5) if this is the best so far, keep it in memory
                        if val_acc > best_val_acc:
                            best_val_acc = val_acc
                            best_epoch   = epoch
                            # deep‐copy the state dict so it's not overwritten
                            best_state   = {k: v.cpu().clone() for k,v in model.state_dict().items()}
                    
                    # After the loop:
                    
                    # — reload best‐validation weights
                    model.load_state_dict(best_state)
                    model.to(device)
                    print(f"Loaded best model from epoch {best_epoch} (val_acc {best_val_acc:.3%})")
                    
                    # — now evaluate on your *test* set
                    test_acc, test_metrics = evaluate_model(model, test_loader, device)
                    print(f"Test Overall Acc: {test_acc:.3%}")
                    print(f"Test Unfiltered Acc: {test_metrics['unfiltered']:.3%}")
                    print(f"Test Filtered   Acc: {test_metrics['filtered']:.3%}")
                    
                    # — log into your results list
                    results.append({
                        'feature_type':    feat_type,
                        'dataset_root':    root,
                        'frequency':       freq,
                        'model':           model_name,
                        'best_val_epoch':  best_epoch,
                        'best_val_acc':    best_val_acc,
                        'test_overall_acc':test_acc,
                        'test_unfiltered_acc': test_metrics['unfiltered'],
                        'test_filtered_acc':   test_metrics['filtered']
                    })
                    print(f"Done {feat_type}@{root}@{freq}Hz with {model_name}")

    # save results
    df = pd.DataFrame(results)
    df.to_csv('difficulty_analysis_results.csv', index=False)
    print("Saved results to difficulty_analysis_results.csv")

Train  filtered: 1503 unfiltered: 1519
Valid  filtered: 191 unfiltered: 175
Total training files found: 3022
Total val files found: 366


Training: 100%|██████████████████████████████████████████████████████████████████████| 756/756 [01:25<00:00,  8.85it/s]


Running Loss 13.725357132613974


Eval: 100%|████████████████████████████████████████████████████████████████████████████| 92/92 [00:07<00:00, 12.82it/s]


Epoch 1 — val_acc: 9.836%


...

Running Loss 0.057174387736637695


Eval: 100%|█████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 201.17it/s]


Epoch 50 — val_acc: 98.882%
Loaded best model from epoch 34 (val_acc 99.281%)


Eval: 100%|█████████████████████████████████████████████████████████████████████████| 322/322 [00:01<00:00, 191.65it/s]

Test Overall Acc: 99.067%
Test Unfiltered Acc: 98.760%
Test Filtered   Acc: 99.376%
{'feature_type': 'MFCC', 'dataset_root': 'D:\\Aliasing3\\Processed_Files\\DS_U8K_MFCC', 'frequency': 22000, 'model': 'CNN', 'overall_acc': 0.24590164422988892, 'unfiltered_acc': 0.2914285659790039, 'filtered_acc': 0.2041884809732437}
Done MFCC@D:\Aliasing3\Processed_Files\DS_U8K_MFCC@22000Hz with CNN
Saved results to difficulty_analysis_results.csv
